-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

<div style="text-align: justify">

# 02 - Clustering K-medias y Conglomerados Jerárquicos
por [Ignacio Sarmiento Barbieri](https://ignaciomsarmiento.github.io/) y [Lucas Gómez Tobón]()

## 1. Contexto del aprendizaje no supervisado
En otras clases probablemente habrá aprendido metodologías para predecir los resultados de una variable $Y=(Y_1, Y_2, \cdots, Y_N)$ a partir de un conjunto de predictores $X^T=(X_1, X_2, \cdots, X_p)$. Típicamente a este tipo de ejercicios se les conoce con el nombre de "aprendizaje supervisado" o "aprendizaje con profesor" pues metafóricamente el estudiante presenta una respuesta $\hat{y_i}$ para cada $x_i$ en la muestra de entrenamiento y el profesor revela la respuesta correcta y el error asociado a la respuesta del estudiante. Este error se caracteriza con una función de pérdida $L(y, \hat{y})$ como por ejemplo: $L(y, \hat{y}) = (y-\hat{y})^2$.

Suponiendo que $(X, Y)$ son variables aleatorias representadas por alguna función de densidad conjunta $Pr(X,Y)$, el aprendizaje supervisado puede ser formalmente caracterizado como un problema de estimación de densidad en donde nos interesa enfocarnos en las propiedades de la densidad condicional $Pr(Y|X)$. 
$$Pr(X,Y)=Pr(Y|X)\cdot Pr(X)$$
En donde $Pr(X)$ es la distribución marginal conjunta de los valores de $X$. En el caso del aprendizaje supervisado la $Pr(X)$ no es de mayor importancia. Sin embargo, para los problemas de esta clase, los problemas de aprendizaje no supervisado, nuestro objetivo será inferir directamente las propiedades de $Pr(X)$ sin la ayuda de un profesor o supervisor que señale la respuesta correcta o el grado de error para cada observación $i$. Por tal motivo, es difícil cuantificar la calidad de los resultados de estos métodos.

## 2. Análisis de clústeres.
El análisis de clústeres (también llamado segmentación de datos) es una de las principales aplicaciones de los algoritmos de aprendizaje no supervisado y consiste dividir las observaciones de un conjunto en un número $m$ de grupos de tal manera que todos los puntos dentro de un mismo grupo estén más estrechamente relacionados entre sí que los puntos asignados a diferentes grupos. 

Note que un concepto central en todo análisis de clústeres es la noción de similaridad o disimilaridad entre observaciones la cual podrá variar según las decisiones del desarrollador. Existen algunas guías para escoger una adecuada medida de distancia, sin embargo, existe un buen grado de subjetividad en la escogencia de estas (de algún modo esto se asemeja a la elección de la función de pérdida para el caso de aprendizaje supervisado). Por tal motivo, echaremos un vistazo sobre los fundamentos de las métricas de distancia antes de pasar por los algoritmos en sí.

## 3. Medidas de distancia o disimilaridad.
### 3.1. Matrices de proximidad o disimilaridad.
Muchas veces los datos son presentados directamente en términos de proximidad entre pares de objetos. Este tipo de datos generalmente se representa en una matriz $D$ de tamaño $N\times N$ en donde $N$ es el número de objetos y cada elemento $d_{ij}$ corresponde a la proximidad entre el objeto $i$ y el objeto $j$. En muchos casos esta matriz se usa como entrada en los algoritmos de clustering.

La mayoría de algoritmos suponen que la matriz de disimilaridad debe ser no negativa y con ceros en la diagonal (es decir que la distancia de un objeto a si mismo es 0). En todo caso, se pueden transformar las medidas de disimilaridad a similaridad (o viceversa) a partir de una transformación monótona creciente (decreciente). Adicionalmente es usual que se suponga que la matriz $D$ sea simétrica, por lo que si originalmente esta no lo es, se puede reemplazar por $(D+D^T)/2$.

En un sentido estricto, las disimilaridades son rara vez una medida de distancia pues no satisfacen la desigualdad triangular $d_{ij}\leq d_{ik} + d_{jk}$, para todo $k\in\{1, \cdots, N\}$. Por tal motivo, algunos algoritmos no permiten usar esta matriz como entrada.
### 3.2. Disimilaridad basada en atributos.
Más a menudo nos encontramos que cada objeto $X_{i}$ es medido en más de una variable $j=1,2,\cdots,p$. Por ende, para estimar la matriz de disimilaridad se debe encontrar primero la distancia entre pares de observaciones. 

Para tal fin definimos la disimilaridad $d_j(x_{ij}, x_{i'j})$ entre los valores del atributo $j$ y luego la disimilaridad entre objetos $i$ y $i'$ como:
$$D(x_i,x_j)=\sum_{j=1}^p d_j(x_{ij}, x_{i'j})$$

Entre las medidas de distancia más comunes se encuentra la distancia al cuadrado: $d_j(x_{ij}, x_{i'j}) = (x_{ij} - x_{i'j})^2$. No obstante, otras opciones son posibles y son potenciales de encontrar resultados diferentes. Para atributos no cuantitativos (como las variables categóricas) la distancia al cuadrado no es apropiada. Adicionalmente, en algunos contextos es deseable otorgarle un peso diferenciado a cada uno de los atributos a la hora de computar la distancia.

A continuación se muestran otro tipo de distancias según el tipo de variables:
#### Para variables cuantitativas. 
Las variables cuantitativas son aquellas en las que su dominio es un continuo de números reales. Es usual definir el error entre ellas como una función monótona creciente del valor absoluto de su diferencia:
$$d(x_i, x_{i'})=l(|x_i, x_{i'}|)$$
Alternativamente, los algoritmos de clustering también pueden estar basados en la correlación:
$$\rho(x_i, x_{i'})=\frac{\sum_j (x_{ij}-\bar{x_i})(x_{i'j}-\bar{x_{i'}})}{\sqrt{\sum_j (x_{ij}-\bar{x_i})^2\sum_j(x_{i'j}-\bar{x_{i'}}^2)}}$$
En donde $\bar{x_i}=\sum_j x_{ij}/p$. Note que el promedio es sobre las variables y no sobre las observaciones. Si se estandarizan antes las observaciones, entonces $\sum_j (x_{ij}-x_{i'j})^2 \propto 2(1-\rho(x_i, x_{i'}))$. Por esto los clusteres basados en la correlación (similaridad) son equivalentes a los clusteres basados en la distancia al cuadrado (disimilaridad).

#### Para variables ordinales.
Este tipo de variables usualmente se representa como un continuo de enteros y el dominio de estos es considerado un conjunto ordenado. Las medidas de error para variables ordinales son generalmente definidas reemplazando su valor $M$ original con:
$$\frac{i-1/2}{M},\ i=1,\cdots,M$$
En el orden prescrito de sus valores originales. Luego se tratan como variables cuantitativas en esta escala.

#### Para variables categóricas.
Para variables categóricas no ordenadas (también llamadas variables nominales), el grado de diferencia entre pares de observaciones debe ser determinado de forma explicita. Si la variable tiene $M$ valores posibles, estos pueden ser ordenados en una matriz no negativa, simétrica y con diagonal cero de tamaño $M\times M$. La elección más común es tener $L_{rr'} = 1$ para todo $r\neq r'$ aunque pesos diferenciados pueden ser usados para enfatizar que unos errores pesan más que otros.

### 3.3. Disimilaridad entre objetos.
Ahora definimos una forma para combinar las $p$ medidas de disimilaridades individuales entre atributos $d_j(x_{ij}, x_{i'j})$ en una sola medida de disimilaridad entre objetos $D(x_i, x_{i'})$. Esto casi siempre a partir de un promedio ponderado o combinación convexa:
$$D(x_i, x_{i'})=\sum_{j=1}^p\omega_j\cdot d_j(x_{ij}, x_{i'j});\ \sum_{j=1}^p\omega_j=1$$
Note que $\omega_j$ es el peso asignado al atributo j-ésimo. La escogencia de estos pesos partirá enteramente de criterios subjetivos.
Además es importante recalcar que pesos iguales no necesariamente conlleva a que todos los atributos tengan la misma influencia, por ejemplo $\omega_j=1\ \forall j$. La influencia del atributo j-ésimo $X_j$ en la disimilaridad entre dos objetos depende en su contribución relativa al promedio de disimilaredes entre todos los puntos de un conjunto de datos:
$$\bar{D}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N D(x_i, x_{i'})=\sum_{j=1}^p\omega_j \cdot \bar{d_j}$$
Con $\bar{d_j}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N d_j(x_{ij}, x_{i'j})$ siendo la disimilaridad promedio del atributo $j$. Por consiguiente, la influencia relativa de la variable j-ésima es $\omega_j \cdot \bar{d_j}$ y definiendo $\omega_j \sim 1/\bar{d_j}$ se consigue que todos los atributos aporten lo mismo a la disimilaridad.

Por ejemplo, si se tienen $p$ variables cuantitativas y se define el error al cuadrado como medida de distancia para cada coordinada, obtenemos la distancia euclidiana al cuadrada ponderada por pesos:
$$D_I(x_i, x_{i'})=\sum_{j=1}^p\omega_j\cdot (x_{ij}-x_{i'j})^2$$

Note que $\bar{d_j}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N (x_{ij}-x_{i'j})^2=2\cdot var_j$ en donde $var_j$ es la varianza muestral de $X_j$. Luego, la importancia relativa de cada variable es proporcional a la varianza de esta en el conjunto de datos. <font color='red'> Esto es tan bonito que podríamos sacarlo de acá y ponerlo en un quiz/problem set o algo así</font>.

<font color='red'> Nota para mi yo del futuro. Hacer ejemplo con datos generados para mostrar el problema de poner todos los features con el mismo peso! pg. 525 del Elements</font>.

## 4. Tipos de clustering.
En términos generales podemos dividir los algoritmos de clustering en tres categorías:

- **Algoritmos combinatorios**: estos trabajan directamente sobre los datos sin suponer ninguna distribución generadora de la información.

- **Algoritmos de mezclas**: estos suponen que los datos son una muestra $i.i.d.$ de alguna población descrita por una función de densidad. El mecanismo generador de los datos se ajusta a partir del enfoque de Máxima Verosimilitud o algún otro enfoque bayesiano.

- **Algoritmos de busqueda de moda**: estos adoptan una perspectiva no paramétrica, intentando estimar directamente modos distintos de la función de densidad de probabilidad. Las observaciones "más cercanas" a cada modo respectivo luego definen los grupos individuales.

# K-Medias
El algoritmo de k-medias es un algoritmos de clustering de descenso iterativo. Este se usa cuando todas las variables en el conjunto de datos son numéricas y se utiliza la distancia euclidiana cuadrática para definir la disimilaridad entre observaciones:
$$d(x_i, x_{i'})=\sum_{j=1}^p(x_{ij}-x_{i'j})^2=||x_i-x_{i'}||^2$$
En este caso particular la función de pérdida a minimizar para garantizar que los puntos más cercanos entre sí estén dentro de un mismo segmento es:
$$W(C)=\frac{1}{2}\sum_{k=1}^K\sum_{C(i)=k}\sum_{C(i')=k}||x_i-x_{i'}||^2$$
$$=\sum_{k=1}^K N_k\sum_{C(i)=k}||x_i-\bar{x_{k}}||^2$$
En donde $\bar{x_{k}}=(\bar{x_{1k}}, \cdots, \bar{x_{pk}})$ es el vector de medias asociado al clúster k-ésimo y $N_k=\sum_{i=1}^N(C(i)=k)$.
Por tal motivo, el criterio se minimiza asignando las $N$ observaciones a los $K$ conglomerados de tal manera que dentro de cada conglomerado se minimiza la disimilitud promedio de las observaciones a la media del conglomerado, según lo definido por los puntos en ese conglomerado.
El algoritmo de descenso iterativo para resolver:
$$C^*=\min_C\sum_{k=1}^K N_k \sum_{C(i)=k} ||x_i-\bar{x_k}||^2$$
Puede ser obtenido entendiendo que para cualquier subconjunto de observaciones $S$:

$$
    \bar{x_s}=\argmin_m \sum_{i\in S} ||x_i - m||^2
$$ 

Por tanto, podemos obtener $C^∗$ resolviendo el problema de optimización ampliado:
$$ \min_{C, \{m_k\}_1^K} \sum_{k=1}^K N_k \sum_{C(i)=k} ||x_i-\bar{m_k}||^2 $$

Para minimizar la expresión pasada se usa el siguiente algoritmo: 
1. Para una determinada asignación de conglomerados $C$, la varianza total de los conglomerados se minimiza con respecto a $\{m_1, \cdots , m_K\}$ obteniendo las medias de los conglomerados asignados actualmente $\bar{x_s}=\argmin_m \sum_{i\in S} ||x_i - m||^2$.
2. Dado un conjunto actual de medias $\{m_1, \cdots , m_K\}$, la varianza total de los conglomerados se minimiza asignando cada observación a la media del conglomerado más cercana. Es decir,
$$C(i)=\argmin_{1\leq k \leq K} ||x_i-m_k||^2$$
3. Se repiten los pasos 1 y 2 hasta que las asignaciones no cambien.

Cada uno de los pasos 1 y 2 reduce el valor de la varianza total de los conglomerados, por lo que la convergencia está asegurada. Sin embargo, el resultado puede representar un mínimo local subóptimo. Por ende se recomienda iniciar el algoritmo con varias opciones aleatorias diferentes para las medias iniciales y elegir la solución que tenga el valor más pequeño de la función objetivo.

# Clustering jerárquico
Los resultados del algoritmo de K-medias dependen de la elección del número de clústeres que se buscarán y de la posición inicial de los centroides. En cambio, los métodos de agrupación jerárquica no requieren tales especificaciones. Estos por su parte necesitan que el usuario especifique una medida de disimilitud entre *grupos* de observaciones, basándose en las disimilitudes por pares entre las observaciones en los dos grupos. Como sugiere el nombre, producen representaciones jerárquicas en las que los grupos en cada nivel de la jerarquía se crean fusionando grupos en el siguiente nivel inferior. En el nivel más bajo, cada grupo contiene una única observación. En el nivel más alto, solo hay un grupo que contiene todos los datos.

Las estrategias para la segmentación jerárquica se dividen en dos: aglomerativo (de abajo hacia arriba) y divisivo (de arriba hacia abajo). Las estrategias de aglomeración comienzan en la parte inferior y en cada nivel fusionan recursivamente un par seleccionado de grupos en un solo grupo. Esto produce una agrupación en el siguiente nivel superior con un grupo menos. La pareja elegida para la fusión consta de los dos grupos con la menor disimilitud intergrupal. Los métodos de división comienzan en la parte superior y en cada nivel dividen de forma recursiva uno de los grupos existentes en ese nivel en dos grupos nuevos. La división se elige para producir dos nuevos grupos con la mayor disimilitud entre grupos. Con ambos paradigmas, hay $N - 1$ niveles en la jerarquía.

Cada nivel de la jerarquía representa una agrupación particular de los datos en grupos de observaciones disjuntos. Toda la jerarquía representa una secuencia ordenada de tales agrupaciones. Depende del usuario decidir qué nivel (si lo hay) realmente representa un agrupamiento "natural" en el sentido de que las observaciones dentro de cada uno de sus grupos son suficientemente más similares entre sí que las observaciones asignadas a diferentes grupos en ese nivel. El estadístico Gap se puede utilizar para este propósito.

La división / aglomeración binaria recursiva se puede representar mediante un árbol. Los nodos de los árboles representan grupos. El nodo raíz representa el conjunto de datos completo. Los $N$ nodos terminales representan cada uno una de las observaciones individuales (clústeres singleton). Cada nodo no terminal ("padre") tiene dos nodos hijos. Para la agrupación divisiva, los dos hijos representan los dos grupos resultantes de la división del padre; para la agrupación aglomerativa, los hijos representan los dos grupos que se fusionaron para formar el padre.

La mayoría de los métodos de aglomeración y algunos de división (cuando se ven de abajo hacia arriba) poseen una propiedad de monotonicidad. Es decir, la disimilitud entre los clústeres fusionados es monótona y aumenta con el nivel de la fusión. Por lo tanto, el árbol se puede trazar de manera que la altura de cada nodo sea proporcional al valor de la disimilitud intergrupal entre sus dos hijos. Los nodos terminales que representan observaciones individuales se trazan todos a una altura cero. Este tipo de visualización gráfica se llama dendrograma.

Un dendrograma proporciona una descripción completa altamente interpretable de la agrupación jerárquica en un formato gráfico. Esta es una de las principales razones de la popularidad de los métodos de agrupación jerárquica

## Clustering jerárquico aglomerativo
Los algoritmos de agrupamiento aglomerativo comienzan con cada observación que representa un grupo único. En cada uno de los $N -1$ pasos, los dos grupos más cercanos (menos diferentes) se fusionan en un solo grupo, produciendo un grupo menos en el siguiente nivel superior. Por lo tanto, se debe definir una medida de disimilitud entre dos conglomerados (grupos de observaciones).

Definamos $G$ y $H$ como dos grupos. La disimilaridad entre ambos $d(G,H)$ se calcula a partir del conjunto de disimilitudes de observación por pares $d_{ii'}$ donde un miembro del par $i$ está en $G$ y el otro $i'$ está en $H$. La agrupación aglomerativa de **Enlace Simple (Single Linkage - SL)** toma la disimilitud intergrupal a partir del par más cercano (menos diferente):
$$d_{SL}(G, H)=\min_{i\in G,\ i'\in H} d_{ii'}$$

Esto también se denomina a menudo la técnica del vecino más cercano. La agrupación aglomerativa de **Enlace Completo (Complete Linkage - CL)** (técnica del vecino más lejano) toma la disimilitud entre grupos como la del par más lejano (más diferente):
$$d_{CL}(G, H)=\max_{i\in G,\ i'\in H} d_{ii'}$$

La agrupación en clústeres de **Promedio de Grupo (Group Average - GA)** utiliza la disimilitud promedio entre los grupos:
$$d_{GA}(G, H)=\frac{1}{N_G N_H}\sum_{i\in G}\sum_{i'\in H}d_{ii'}$$

donde $N_G$ y $N_H$ son los números respectivos de observaciones en cada grupo. Aunque ha habido muchas otras propuestas para definir la disimilitud intergrupal en el contexto de la agrupación aglomerativa, las tres anteriores son las más utilizadas.

Si las disimilaridades en los datos $\{d_{ii'}\}$ exhiben una fuerte tendencia a agruparse, con cada uno de los grupos siendo compacto y bien separado de los demás, entonces los tres métodos producirán resultados similares. Los clústeres son compactos si todas las observaciones dentro de ellos están relativamente cerca (pequeñas diferencias) en comparación con las observaciones en diferentes conglomerados. En la medida en que este no sea el caso, los resultados serán diferentes.

El Enlace Simple (SL) solo requiere que una única disimilitud $d_{ii'}$, $i\in G$ e $i'\in H$, sea pequeña para que dos grupos $G$ y $H$ se consideren cercanos, independientemente de las distancias entre el resto de observaciones. Por lo tanto este tipo de medida tendrá una tendencia a combinar, en umbrales relativamente bajos, observaciones vinculadas por una serie de observaciones intermedias cercanas. Este fenómeno denominado encadenamiento a menudo se considera un defecto del método. Los grupos producidos por el Enlace Simple (SL) pueden violar entonces la propiedad de "compacidad" la cual consiste en que todas las observaciones dentro de cada grupo deben ser similares entre sí, en función de las diferencias de observación proporcionadas $\{d_{ii'}\}$. Si definimos el diámetro $D_G$ de un grupo de observaciones como la mayor disimilitud entre sus miembros
$$D_G=\max_{i\in G,\ i'\in G}d_{ii'}$$
Entonces podemos afirmar que el Enlace Simple (SL) puede producir clústeres con un diametro muy grande.

El Enlace Completo o Vinculación Completa (CL) representa el caso opuesto. Dos grupos $G$ y $H$ se consideran cercanos solo si todas las observaciones de la unión son relativamente similares. Esto tiende a producir clústeres compactos con diametros pequeños. Sin embargo, este método puede producir agrupaciones que violen la propiedad de "cercanía". Esto es que las observaciones que se asignen a un clúster pueden ser más similares a algunas observaciones de otros clústeres en comparación con algunas observaciones de su propia agrupación.

El agrupamiento Promedio de Grupo (GA) representa un intermedio entre los dos extremos del enlace simple y completo. Intenta producir agrupaciones relativamente compactas que están relativamente alejadas. Sin embargo, sus resultados dependen de la escala numérica en la que se miden las diferencias de observación $d_{ii'}$. La aplicación de una transformación monótona estrictamente creciente $h(·)$ a $d_{ii'}$, $h_{ii'}= h(d_{ii'})$, puede cambiar el resultado producido por el enlace Promedio de Grupo. En contraste, los otros tipos de enlaces vistos dependen solo del orden de $d_{ii'}$ y, por lo tanto, son invariantes a tales transformaciones monótonas. Esta invariancia se utiliza a menudo como un argumento a favor de la vinculación simple o completa sobre los métodos de promedio de grupo.

## Clustering jerárquico divisivo
Los algoritmos de agrupación en clústeres divisivos comienzan con todo el conjunto de datos como un único clúster y dividen de forma recursiva uno de los clústeres existentes en dos clústeres secundarios en cada iteración de forma descendente. En el contexto de segmentación, una ventaja potencial de la división sobre los métodos aglomerativos puede ocurrir cuando el interés se centra en dividir los datos en un número relativamente pequeño de agrupaciones.

El paradigma divisivo se puede emplear aplicando recursivamente cualquiera de los métodos combinatorios como K-medias o K-medoides, con $K = 2$, para realizar las divisiones en cada iteración. Sin embargo, tal enfoque dependería de la configuración inicial especificada en cada paso. Además, no produciría necesariamente una secuencia de división que posea la propiedad de monotonicidad requerida para la representación de un dendrograma.

Macnaughton Smith et al. (1965). Propusieron un algoritmo divisivo que evita estos problemas. Comienza colocando todas las observaciones en un solo grupo $G$. Luego se elige la observación cuya diferencia promedio a todas las demás observaciones sea mayor. Esta observación forma el primer miembro de un segundo grupo $H$. En cada paso sucesivo, esa observación en $G$ cuya distancia promedio a las de $H$, menos la de las observaciones restantes en $G$ es mayor, se transfiere a $H$. Esto continúa hasta la diferencia correspondiente en promedios se vuelva negativa. Es decir, ya no hay observaciones en $G$ que estén, en promedio, más cerca de las de $H$. El resultado es una división del grupo original en dos grupos hijos, las observaciones que se transfieren a $H$ y las que se quedan en $G$. Estos dos grupos representan el segundo nivel de la jerarquía. Cada nivel sucesivo se produce aplicando este procedimiento de división a uno de los grupos del nivel anterior. Kaufman y Rousseeuw (1990) sugieren elegir el grupo en cada nivel con el diámetro más grande para la división. Otra alternativa sería elegir el grupo que tenga la mayor disimilitud promedio entre sus miembros.
$$\bar{d_G}=\frac{1}{N_G^2}\sum_{i\in G}\sum_{i'\in G} d_{ii'}$$
La partición recursiva continua hasta que todos los clústeres se vuelvan singletons o todos los miembros de cada segmento tengan disimilaridad cero entre ellos.

## Recursos

Hastie, T., Tibshirani, R., & Friedman, J. H. (2001). The elements of statistical learning: Data mining, inference, and prediction. New York: Springer.

Kaufman, L. & Rousseeuw, P. (1990). Finding Groups in Data: An Introduction to Cluster Analysis, Wiley, New York.

Macnaughton Smith, P., Williams, W., Dale, M. & Mockett, L. (1965). Dissimilarity analysis: a new technique of hierarchical subdivision, Nature 202: 1034–1035.